In [1]:
!pip install requests
!pip install pandas
!pip install PyMuPDF
!pip install pytesseract
!pip install openpyxl
!pip install requests
!pip install random


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 65.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random


In [ ]:
# 1) Install deps
!pip -q install camelot-py[cv] pandas
!apt -yqq install ghostscript >/dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.0 MB/s eta 0:00:00




Bulk Downloader

In [2]:
import os
import requests
from datetime import datetime, timedelta

# URL pattern
url_base = "https://dps.psx.com.pk/download/closing_rates/"

# Function to generate the URL from a date
def generate_url(date):
    return f"{url_base}{date.strftime('%Y-%m-%d')}.pdf"

# Function to check if the date is a weekend (Saturday or Sunday)
def is_weekend(date):
    return date.weekday() >= 5  # 5 for Saturday, 6 for Sunday

# Create a directory to save the downloaded files
output_dir = "/content/closing_rates_pdfs"
os.makedirs(output_dir, exist_ok=True)

# Function to download a file with error handling
def download_file(url, date):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            file_name = f"{date.strftime('%Y-%m-%d')}.pdf"
            with open(os.path.join(output_dir, file_name), 'wb') as file:
                file.write(response.content)
            print(f"Downloaded {file_name}")
        else:
            print(f"Failed to download {url} - Status Code: {response.status_code}")
    except Exception as e:
        print(f"Error downloading {url}: {str(e)}")

# Function to handle bulk downloads with rate-limiting and skipping weekends
def download_files(start_date, end_date):
    current_date = start_date
    while current_date <= end_date:
        if not is_weekend(current_date):  # Skip weekends
            url = generate_url(current_date)
            download_file(url, current_date)
        else:
            print(f"Skipped weekend: {current_date.strftime('%Y-%m-%d')}")
        current_date += timedelta(days=1)  # Move to the next day
    print("Download complete.")

# Example to start downloading from 2020-08-03 to 2020-08-07
start_date = datetime(2020, 8, 3)
end_date = datetime(2025, 8, 29)
download_files(start_date, end_date)


Downloaded 2020-08-03.pdf
Downloaded 2020-08-04.pdf
Downloaded 2020-08-05.pdf
Downloaded 2020-08-06.pdf


KeyboardInterrupt: 

In [ ]:
# 2) Convert all PDFs to CSVs (and make one master CSV)
import os
import re
from datetime import datetime
import pandas as pd
import camelot

PDF_DIR = "/content/closing_rates_pdfs"
CSV_DIR = "/content/closing_rates_csv"
os.makedirs(CSV_DIR, exist_ok=True)

def extract_date_from_filename(fname):
    # expects YYYY-MM-DD.pdf
    stem = os.path.splitext(os.path.basename(fname))[0]
    try:
        return datetime.strptime(stem, "%Y-%m-%d").date()
    except Exception:
        return None

all_rows = []   # to build the master CSV

pdf_files = sorted([f for f in os.listdir(PDF_DIR) if f.lower().endswith(".pdf")])
if not pdf_files:
    print("No PDFs found in", PDF_DIR)

for fname in pdf_files:
    fpath = os.path.join(PDF_DIR, fname)
    date_val = extract_date_from_filename(fname)

    # Try lattice first (ruled tables). If empty, try stream.
    try:
        tables = camelot.read_pdf(fpath, pages="all", flavor="lattice", strip_text="\n")
    except Exception:
        tables = []

    if not tables or len(tables) == 0 or sum(t.shape[0] for t in tables) == 0:
        tables = camelot.read_pdf(fpath, pages="all", flavor="stream", strip_text="\n")

    if not tables or len(tables) == 0:
        print(f"[WARN] No tables found in {fname}")
        continue

    # Concatenate all tables from this PDF
    pdf_df = pd.concat([t.df for t in tables], ignore_index=True)

    # Heuristic cleanups:
    # - Make the first row headers if it looks like headers
    # - Drop completely empty columns/rows
    pdf_df.replace(r"^\s*$", pd.NA, regex=True, inplace=True)
    pdf_df.dropna(how="all", axis=0, inplace=True)
    pdf_df.dropna(how="all", axis=1, inplace=True)

    # Promote header row if it contains typical column labels
    header_candidates = ["Company", "Company Name", "Turnover", "Prv.Rate", "Open", "Highest", "Lowest", "Last", "Rate", "Diff"]
    if any(any(isinstance(x, str) and hc.lower() in x.lower() for x in pdf_df.iloc[0].tolist()) for hc in header_candidates):
        pdf_df.columns = pdf_df.iloc[0].astype(str).str.strip()
        pdf_df = pdf_df.iloc[1:].reset_index(drop=True)

    # Normalize common column names
    def normalize(col):
        c = str(col).strip()
        c = re.sub(r"\s+", " ", c)
        c = c.replace("Prv.", "Prev").replace("Last Rate", "Last").replace("Open Rate", "Open")
        return c
    pdf_df.columns = [normalize(c) for c in pdf_df.columns]

    # Add Date column
    if date_val is not None:
        pdf_df.insert(0, "Date", date_val)

    # Save per-PDF CSV
    out_csv = os.path.join(CSV_DIR, fname.replace(".pdf", ".csv"))
    pdf_df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"[OK] {fname} -> rows: {len(pdf_df):>5} | saved: {out_csv}")

    # Append to master (only rows that look like data: have at least 3 non-NA cells)
    mask_data = pdf_df.notna().sum(axis=1) >= 3
    all_rows.append(pdf_df[mask_data])

# Build and save master CSV
if all_rows:
    master = pd.concat(all_rows, ignore_index=True)
    master_out = "/content/psx_closing_rates_master.csv"
    master.to_csv(master_out, index=False, encoding="utf-8-sig")
    print(f"\n[MASTER] Combined rows: {len(master)}")
    print(f"[MASTER] Saved: {master_out}")
else:
    print("\n[MASTER] No rows extracted — check a sample PDF formatting or try changing flavor to 'stream' only.")



In [ ]:
import os, glob
pdfs = sorted(glob.glob("/content/closing_rates_pdfs/*.pdf"))
print(f"Found {len(pdfs)} PDFs")
print("\n".join(os.path.basename(p) for p in pdfs[:10]))


In [ ]:
from google.colab import files
import zipfile

# Create a zip file for easier download
zip_file = "/content/closing_rates_pdfs.zip"

# Zip the directory
with zipfile.ZipFile(zip_file, 'w') as zipf:
    for root, dirs, files in os.walk(output_dir):
        for file in files:
            zipf.write(os.path.join(root, file), file)

# Provide the zip file for download
files.download(zip_file)


In [ ]:
from google.colab import files
import zipfile

# Create a zip file for easier download
zip_file = "/content/closing_rates_csv.zip"

# Zip the directory
with zipfile.ZipFile(zip_file, 'w') as zipf:
    for root, dirs, files in os.walk(output_dir):
        for file in files:
            zipf.write(os.path.join(root, file), file)

# Provide the zip file for download
files.download(zip_file)


Delete Files

In [ ]:
import shutil

# Replace the path with the directory you want to delete
shutil.rmtree('/content/closing_rates_pdfs/')
shutil.rmtree('/content/closing_rates_csv/')
